In [1]:
groq_api_key = "gsk_k7pKrTS*********************aO1GULgPR"

In [2]:
from langchain_groq import ChatGroq

In [3]:
model = ChatGroq(api_key=groq_api_key , model="llama3-70b-8192" , temperature=0)

In [4]:
model.invoke('hii')

AIMessage(content="Hii! How's it going?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 12, 'total_tokens': 21, 'completion_time': 0.027768569, 'prompt_time': 0.000134648, 'queue_time': 0.054331119, 'total_time': 0.027903217}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_dd4ae1c591', 'finish_reason': 'stop', 'logprobs': None}, id='run-ea8a0470-9f52-4579-b426-26e4c5802a8f-0', usage_metadata={'input_tokens': 12, 'output_tokens': 9, 'total_tokens': 21})

## defining tools

In [6]:
from langchain_core.tools import tool,StructuredTool
import yfinance as yf

In [7]:
@tool
def company_address(ticker:str)-> str:
    """Return company address for input ticker.
    eg. company_address : AAPL
    retrun company address for ticker AAPL which is stock ticker for apple INC"""
    ticker_obj = yf.Ticker(ticker)
    info = ticker_obj.get_info()
    return " ".join([info[key] for key in ['address','city','state','zip','country']]) 

In [8]:
@tool
def full_time_employee(ticker:str)-> str:
    """return full time employee count for input ticker
    eg full_time_employee : MSFT
    return full time employee count for ticker msft which is stock ticker in microsoft
    """
    ticker_obj = yf.Ticker(ticker)
    info = ticker_obj.get_info()
    return info['fullTimeEmployees']

In [9]:
@tool
def last_close_price(ticker:str)-> float:
    """Return last count price for input ticker 
    eg last_count_price : AAPL
    return last count price of AAPL which is stock ticker for apple inc"""
    ticker_obj = yf.Ticker(ticker)
    info = ticker_obj.get_info()
    return info['previousClose']

In [10]:
@tool
def EBITDA(ticker:str) -> float:
    """return EBITDA for input ticker
    eg EBITDA : MSFT
    return EBITDA of MSFT which is stock ticker for microsoft
    """
    ticker_obj = yf.Ticker(ticker)
    info = ticker_obj.get_info()
    return info['ebitda']

In [11]:
@tool
def total_debt(ticker :str)-> float:
    """ Return total debt for input ticker
    eg total_debt : AAPL
    return total debt of AAPL which is stock ticker for apple
    """
    ticker_obj = yf.Ticker(ticker)
    info = ticker_obj.get_info()
    return info['totalDebt']

In [12]:
@tool
def total_revenue(ticker :str)-> str:
    """ Return total revenue for input ticker
    eg total_revenue : AAPL
    return total revenue of AAPL which is stock ticker for apple
    """
    ticker_obj = yf.Ticker(ticker)
    info = ticker_obj.get_info()
    return info['totalRevenue']

In [13]:
@tool
def debt_to_equility(ticker :str)-> float:
    """ Return debt to equility for input ticker
    eg debt_to_equility : AAPL
    return debt to equility of AAPL which is stock ticker for apple
    """
    ticker_obj = yf.Ticker(ticker)
    info = ticker_obj.get_info()
    return info['debtToEquity']

In [14]:
tools = [company_address,
         full_time_employee,
         last_close_price,
         EBITDA,  
         total_debt,
         total_revenue,
         debt_to_equility]

In [15]:
llm = model.bind_tools(tools , tool_choice="auto")

In [16]:
llm.invoke('hii?')

AIMessage(content='Hi! How can I help you today?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 1661, 'total_tokens': 1671, 'completion_time': 0.081120559, 'prompt_time': 0.062459281, 'queue_time': 0.060236938999999996, 'total_time': 0.14357984}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_dd4ae1c591', 'finish_reason': 'stop', 'logprobs': None}, id='run-60006f45-75ae-4e92-bf85-82084dc8088e-0', usage_metadata={'input_tokens': 1661, 'output_tokens': 10, 'total_tokens': 1671})

## ReAct agent

In [18]:
from langgraph.graph import StateGraph , END
from typing import TypedDict , Annotated 
import operator
from langchain_core.messages import HumanMessage , SystemMessage , ToolMessage , AnyMessage

In [43]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

In [45]:
class ReActAgent:
    
    def __init__(self, model, tools, system=""):
        self.system = system
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools, tool_choice="auto")
        
        graph = StateGraph(AgentState)
        graph.add_node("llama3", self.call_llm)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges(
            "llama3",
            self.exists_action,
            {True: "action", False: END}
        )
        graph.add_edge("action", "llama3")
        graph.set_entry_point("llama3")
        self.graph = graph.compile()
        
    def exists_action(self, state: AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    def call_llm(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling Tool: {t}")
            if not t['name'] in self.tools: 
                print(f"\n Tool: {t} does not exist.")
                result = "Incorrect Tool Name, Please Retry and Select tool from List of Available tools." 
            else:
                result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Tools Execution Complete. Back to the model!")
        return {'messages': results}

In [47]:
prompt= """
you are a smart ai finance assistant use list of aviable tools to answer question if need .
you are allowed to call multiples calls (either together or a sequence),
if you need to look up some information before asking a follow up question , you are allow to do that."""

In [49]:
agent = ReActAgent(model , tools , system = prompt)

## use agent

In [52]:
messages = [HumanMessage(content="What is EBITDA and debt-to-equity ratio of Google?")]
result = agent.graph.invoke({'messages': messages})
print(result['messages'][-1].content)

Calling Tool: {'name': 'EBITDA', 'args': {'ticker': 'GOOGL'}, 'id': 'call_sh8s', 'type': 'tool_call'}
Calling Tool: {'name': 'debt_to_equility', 'args': {'ticker': 'GOOGL'}, 'id': 'call_x0dt', 'type': 'tool_call'}
Tools Execution Complete. Back to the model!
The EBITDA of Google is 129496997888 and the debt-to-equity ratio is 8.655.


In [56]:
messages = [HumanMessage(content="Compare total revenue of Apple and Nvidia and how many times.")]
result = agent.graph.invoke({'messages': messages})
print(result['messages'][-1].content)

Calling Tool: {'name': 'total_revenue', 'args': {'ticker': 'AAPL'}, 'id': 'call_w805', 'type': 'tool_call'}
Calling Tool: {'name': 'total_revenue', 'args': {'ticker': 'NVDA'}, 'id': 'call_4kx7', 'type': 'tool_call'}
Tools Execution Complete. Back to the model!
To compare the total revenue of Apple and Nvidia, I can calculate the ratio of Apple's total revenue to Nvidia's total revenue.

Apple's total revenue is 395760009216.
Nvidia's total revenue is 130497003520.

The ratio of Apple's total revenue to Nvidia's total revenue is approximately 3.03.

So, Apple's total revenue is approximately 3.03 times Nvidia's total revenue.


In [58]:
messages = [HumanMessage(content="total stocks in tesla")]
result = agent.graph.invoke({'messages': messages})
print(result['messages'][-1].content)

Calling Tool: {'name': 'total_revenue', 'args': {'ticker': 'TSLA'}, 'id': 'call_57yn', 'type': 'tool_call'}
Tools Execution Complete. Back to the model!
Calling Tool: {'name': 'full_time_employee', 'args': {'ticker': 'TSLA'}, 'id': 'call_yczr', 'type': 'tool_call'}
Tools Execution Complete. Back to the model!
Calling Tool: {'name': 'last_close_price', 'args': {'ticker': 'TSLA'}, 'id': 'call_d71z', 'type': 'tool_call'}
Tools Execution Complete. Back to the model!
You want to know the total stocks in Tesla. Based on the information gathered, I don't have a direct tool to provide the total stocks. However, I can provide some relevant information. The total revenue of Tesla is 95724003328, and the full-time employee count is 125665. The last close price of Tesla's stock is 250.74. If you want to know the total stocks, you might need to look up the total shares outstanding or the market capitalization of Tesla, which I don't have a tool for.
